In [1]:
import pybaseball as pyb
from time import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

In [2]:
a = time() # you can uncomment 2023 and 2024 if you want
#df_2023 = pyb.statcast(start_dt='2023-03-30', end_dt='2023-09-29')
#df_2024 = pyb.statcast(start_dt='2024-03-30', end_dt='2024-09-29')
df_2025 = pyb.statcast(start_dt='2025-03-30', end_dt='2025-09-29')
print(time() - a)

This is a large query, it may take a moment to complete


C:\Users\owenc\anaconda3\Lib\site-packages\pybaseball\statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
  0%|          | 0/184 [00:00<?, ?it/s]C:\Users\owenc\anaconda3\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = d

398.51291251182556


In [3]:
#df_2023 = df_2023.iloc[::-1]
#df_2024 = df_2024.iloc[::-1]
df_2025 = df_2025.iloc[::-1]

In [4]:
df = df_2025

In [5]:
first_pitch_per_pitcher = df.groupby(['game_pk', 'pitcher']).head(1)

# Step 2: Starters are those whose first pitch came in inning 1 (top or bottom)
# Warning, this line really takes anyone whose first pitch was in 1st inning and calls them a starter
# So if a starter has a meltdown and a reliever comes in in the 1st, this is still labeling the reliever a starter
starters = first_pitch_per_pitcher[first_pitch_per_pitcher['inning'] == 1]

# Step 3: Create starter ID pairs (game_pk, pitcher)
starter_ids = starters[['game_pk', 'pitcher']].drop_duplicates()

# Step 4: Label each row in the full DataFrame as 'starter' or 'reliever'
df = df.copy()  # avoid SettingWithCopyWarning

is_starter = pd.MultiIndex.from_frame(df[['game_pk','pitcher']]).isin(
    pd.MultiIndex.from_frame(starter_ids)
)
df['role'] = np.where(is_starter, 'starter', 'reliever')


In [6]:
df

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,role
3489,FF,2025-03-30,90.8,-1.35,6.74,"Wacha, Michael",680757,608379,NaN,ball,...,1.01,0.5,-0.5,53.6,<NA>,<NA>,<NA>,<NA>,<NA>,starter
3374,CH,2025-03-30,80.0,-1.56,6.8,"Wacha, Michael",680757,608379,NaN,ball,...,2.37,1.19,-1.19,53.3,<NA>,<NA>,<NA>,<NA>,<NA>,starter
3270,FF,2025-03-30,91.2,-1.52,6.7,"Wacha, Michael",680757,608379,NaN,called_strike,...,1.16,0.61,-0.61,51.1,<NA>,<NA>,<NA>,<NA>,<NA>,starter
3154,FF,2025-03-30,92.2,-1.45,6.64,"Wacha, Michael",680757,608379,NaN,called_strike,...,1.07,0.72,-0.72,49.7,<NA>,<NA>,<NA>,<NA>,<NA>,starter
3086,CH,2025-03-30,81.0,-1.39,6.69,"Wacha, Michael",680757,608379,single,hit_into_play,...,2.51,1.11,-1.11,52.7,7.929151,-13.522106,50.545652,26.249933,27.690894,starter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364,SL,2025-09-28,85.3,-2.26,4.85,"Weissert, Greg",668670,669711,NaN,called_strike,...,2.71,-0.52,-0.52,15.8,<NA>,<NA>,<NA>,<NA>,<NA>,reliever
2261,SL,2025-09-28,84.8,-2.33,4.72,"Weissert, Greg",668670,669711,NaN,swinging_strike,...,2.55,-0.32,-0.32,12.3,13.78541,4.08139,32.414181,38.011685,27.083341,reliever
2185,FF,2025-09-28,95.4,-1.99,5.22,"Weissert, Greg",668670,669711,NaN,foul,...,1.36,0.85,0.85,22.9,2.871131,31.805044,22.266527,37.478847,15.582717,reliever
2090,FF,2025-09-28,95.1,-1.91,5.1,"Weissert, Greg",668670,669711,strikeout,called_strike,...,1.59,0.93,0.93,20.5,<NA>,<NA>,<NA>,<NA>,<NA>,reliever


In [7]:
list(df.columns) # this is a list of all the attributes

['pitch_type',
 'game_date',
 'release_speed',
 'release_pos_x',
 'release_pos_z',
 'player_name',
 'batter',
 'pitcher',
 'events',
 'description',
 'spin_dir',
 'spin_rate_deprecated',
 'break_angle_deprecated',
 'break_length_deprecated',
 'zone',
 'des',
 'game_type',
 'stand',
 'p_throws',
 'home_team',
 'away_team',
 'type',
 'hit_location',
 'bb_type',
 'balls',
 'strikes',
 'game_year',
 'pfx_x',
 'pfx_z',
 'plate_x',
 'plate_z',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'inning',
 'inning_topbot',
 'hc_x',
 'hc_y',
 'tfs_deprecated',
 'tfs_zulu_deprecated',
 'umpire',
 'sv_id',
 'vx0',
 'vy0',
 'vz0',
 'ax',
 'ay',
 'az',
 'sz_top',
 'sz_bot',
 'hit_distance_sc',
 'launch_speed',
 'launch_angle',
 'effective_speed',
 'release_spin_rate',
 'release_extension',
 'game_pk',
 'fielder_2',
 'fielder_3',
 'fielder_4',
 'fielder_5',
 'fielder_6',
 'fielder_7',
 'fielder_8',
 'fielder_9',
 'release_pos_y',
 'estimated_ba_using_speedangle',
 'estimated_woba_using_speedangle',
 'w

In [8]:
df['pitch_count'] = df.groupby(['game_pk', 'pitcher']).cumcount() + 1

In [9]:
df[df['on_3b'].isna()]

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,role,pitch_count
3489,FF,2025-03-30,90.8,-1.35,6.74,"Wacha, Michael",680757,608379,NaN,ball,...,0.5,-0.5,53.6,<NA>,<NA>,<NA>,<NA>,<NA>,starter,1
3374,CH,2025-03-30,80.0,-1.56,6.8,"Wacha, Michael",680757,608379,NaN,ball,...,1.19,-1.19,53.3,<NA>,<NA>,<NA>,<NA>,<NA>,starter,2
3270,FF,2025-03-30,91.2,-1.52,6.7,"Wacha, Michael",680757,608379,NaN,called_strike,...,0.61,-0.61,51.1,<NA>,<NA>,<NA>,<NA>,<NA>,starter,3
3154,FF,2025-03-30,92.2,-1.45,6.64,"Wacha, Michael",680757,608379,NaN,called_strike,...,0.72,-0.72,49.7,<NA>,<NA>,<NA>,<NA>,<NA>,starter,4
3086,CH,2025-03-30,81.0,-1.39,6.69,"Wacha, Michael",680757,608379,single,hit_into_play,...,1.11,-1.11,52.7,7.929151,-13.522106,50.545652,26.249933,27.690894,starter,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364,SL,2025-09-28,85.3,-2.26,4.85,"Weissert, Greg",668670,669711,NaN,called_strike,...,-0.52,-0.52,15.8,<NA>,<NA>,<NA>,<NA>,<NA>,reliever,17
2261,SL,2025-09-28,84.8,-2.33,4.72,"Weissert, Greg",668670,669711,NaN,swinging_strike,...,-0.32,-0.32,12.3,13.78541,4.08139,32.414181,38.011685,27.083341,reliever,18
2185,FF,2025-09-28,95.4,-1.99,5.22,"Weissert, Greg",668670,669711,NaN,foul,...,0.85,0.85,22.9,2.871131,31.805044,22.266527,37.478847,15.582717,reliever,19
2090,FF,2025-09-28,95.1,-1.91,5.1,"Weissert, Greg",668670,669711,strikeout,called_strike,...,0.93,0.93,20.5,<NA>,<NA>,<NA>,<NA>,<NA>,reliever,20


In [10]:
import time

below is the key function I have added. It analyzes a single pitcher's data back a customizable number of pitches, analyzes some running averages, and implements RE288 for him alone.

In [86]:
def analyze(pname, extent): 
    # What this does is it adds a "running average" for a few stats to the dataframe for a specific pitcher across 2025...
    # ... and then find that specific pitcher's RE288 (expected runs given BSO123) average.
    pitcher = pyb.playerid_lookup(pname)['key_mlbam'][0]
    # this, for a single pitcher, gives a running average of certain datapoints over a certain number of pitches in the past
    d = df[df['pitcher']==pitcher] #only this pitcher
    print("Testing...", pname)
    time.sleep(2.0)
    dr = d
    dr['item']=np.array(range(len(d)))
    # dr = d.reindex(np.array(range(len(d)))) #the indices are weird, this keeps it as 0 through n
    # Use existing data to calculate each individual pitcher's RE288
    extent = min(extent, len(dr)) # don't go beyond the number of pitches he actually threw!
    print("Extent = ",extent)
    time.sleep(2.0)
    def rsiti(game, inning, t, curr): #runs scored in this inning, or until the pitcher is yanked.
        u = {0: "Top", 1:"Bot"}
        thisinning = dr[(dr['game_pk']==game) & (dr['inning']==inning) & (dr['inning_topbot']==u[t])] #restricts to an individual inning and pitcher
        return max(thisinning['bat_score']-curr) #how many more runs?
    time.sleep(2.0)
    #print(dr['game_pk'])
    print("Adding the remaining runs datapoint.")
    dr['remaining_runs']=dr.apply(lambda x: rsiti(x.game_pk, x.inning, int(x.inning_topbot=='Bot'), x.bat_score), axis=1) #turns this into a dataframe column
    time.sleep(2.0)
    print("Adding the RE288 datapoint.")
    time.sleep(2.0)
    dr['re288case']=dr.apply(lambda x: 252 - 144 * pd.isna(x['on_3b']) - 72 * pd.isna(x['on_2b']) - 36 * pd.isna(x['on_1b']) + 3*x['balls']+x['strikes']+12*x['outs_when_up'], axis=1
                            ) # turns the BSO123 instance into a number from 0 to 287.
    print("Moving on.")
    time.sleep(2.0)
    re288={i:0 for i in range(288)} # these dicts will calculate the pitcher's personal re288 value
    re288counts={i:0 for i in range(288)}
    running_outs=np.zeros(len(dr)) # running total of outs
    running_7plus = np.zeros(len(dr)) # running total of long at bats (7 or more pitches, assumes no mid-PA pitching changes)
    running_risp = np.zeros(len(dr)) # running total of pitches thrown with runner(s) in scoring position
    running_restarts = np.zeros(len(dr)) # running total of games pitched in 
    scale = np.concatenate((np.array(range( 1, extent+1)), extent*np.ones(len(dr)-extent).astype(int))) # scales the running totals, turns them into averages
    for i in range(len(dr)):
        if i%100==0:
            print(i, "pitches covered...")
            time.sleep(1.0)
        drl = dr.loc[dr['item']==i].iloc[0]
        #print(drl, type(drl))
        if i==0:
            running_outs[i]=0
            running_7plus[i]=0
            running_risp[i]=1-int(pd.isna(drl['on_2b']) & pd.isna(drl['on_3b']))
            running_restarts[i]=int(drl['pitch_count']==1)
            re288counts[int(drl['re288case'])]+=1
            re288[int(drl['re288case'])]+=int(drl['remaining_runs'])
            #print(int(drl['remaining_runs']), "runs", "in position", int(drl['re288case']))
        elif i < extent:
            #print(drl['outs_when_up'],dr.loc[dr['item']==i-1].iloc[0]['outs_when_up'])
            running_outs[i]=running_outs[i-1]+(drl['outs_when_up']-dr.loc[dr['item']==i-1].iloc[0]['outs_when_up'])%3 # add 1 if an out occurred
            running_7plus[i]=running_7plus[i-1]+(drl['pitch_number']==7) # 7thpitch of at bat = trigger
            running_risp[i]=running_risp[i-1]+1-(pd.isna(drl['on_2b']) & pd.isna(drl['on_3b']))
            running_restarts[i]=running_restarts[i-1]+(drl['pitch_count']==1) # pitch count of 1 is the trigger
            re288counts[int(drl['re288case'])]+=1
            re288[int(drl['re288case'])]+=int(drl['remaining_runs'])
            #print(int(drl['remaining_runs']), "runs", "in position", int(drl['re288case']))
        else:
            if i==extent: # throughout this, take out - extent
                merp = 0
            else:
                merp = (dr.loc[dr['item']==i-extent]['outs_when_up']-dr.loc[dr['item']==i-extent-1].iloc[0]['outs_when_up'])%3
            drlx = dr.loc[dr['item']==i-extent].iloc[0]
            running_outs[i]=running_outs[i-1]+(drl['outs_when_up']-dr.loc[dr['item']==i-1].iloc[0]['outs_when_up'])%3-merp
            running_7plus[i]=running_7plus[i-1]+(drl['pitch_number']==7)-(drlx['pitch_number']==7)
            running_risp[i]=running_risp[i-1]-(pd.isna(drl['on_2b']) & pd.isna(drl['on_3b']))+(pd.isna(drlx['on_2b']) & pd.isna(drlx['on_3b']))
            running_restarts[i]=running_restarts[i-1]+(drl['pitch_count']==1)-(drlx['pitch_count']==1)
            re288counts[int(drl['re288case'])]+=1
            re288[int(drl['re288case'])]+=(drl['remaining_runs'])
            #print(int(drl['remaining_runs']), "runs", "in position", int(drl['re288case']))
    print("Completed.")
    time.sleep(2.0)
    dr['ro']=running_outs / scale # this "scale"makes everything a running average
    dr['r7']=running_7plus / scale
    dr['rs']=running_risp / scale
    dr['rr']=running_restarts / scale # four new columns added to dataframe
    re288avgs={}
    for i in range(288):
        if re288counts[i]==0:
            re288avgs[i]= float("nan")
        else:
            re288avgs[i] = re288[i]/re288counts[i] # actual re288 averages
    print("Returning values...")
    time.sleep(2.0)
    dr['re288_expected_runs']=dr. apply(lambda x: re288avgs[int(x['re288case'])], axis=1)
    return dr, re288avgs
    # there must be a WILDLY more efficient way to do this and I haven't tested it, but this is the idea
    

In [ ]:
analyze("Skenes", 100)
# Paul Skenes, Cy Young Frontrunner for the Pirates. With a 2025 era of 1.97 he has unusually low RE288 values.

In [32]:
pyb.playerid_lookup("Skenes")['key_mlbam'][0]

694973

Things to add: distinguish between when a pitcher gets out of the inning and when he is taken out mid-inning?